In [1]:
# Import Libraries

import azureml.core
from azureml.core import Workspace

from azureml.core import Dataset
from azureml.data.datapath import DataPath

import os

In [2]:
# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with testerinos


In [3]:
default_ds = ws.get_default_datastore()

if 'titanic dataset' not in ws.datasets:
    Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'titanic-data/')
                              )

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'titanic-data/titanic.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='titanic dataset',
                                description='titanic data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

2023-02-27 20:16:39.057223 | ActivityCompleted: Activity=get_by_name, HowEnded=Failure, Duration=106.62 [ms], Info = {'activity_id': '948a9ce7-c290-4e46-8659-0155f681ccf2', 'activity_name': 'get_by_name', 'activity_type': 'PublicApi', 'app_name': 'dataset', 'source': 'azureml.dataset', 'version': '1.48.0', 'dataprepVersion': '4.8.4', 'sparkVersion': '', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': '', 'completionStatus': 'Success', 'durationMs': 0.11}, Exception=UserErrorException; UserErrorException:
	Message: Cannot find dataset registered with name "titanic dataset" (version: None) in the workspace.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot find dataset registered with name \"titanic dataset\" (version: None) in the workspace."
    }
}
Failed to extract subscription information, Exception=AttributeError; 'Logger' object has no attribute 'activity_info

Validating arguments.
Arguments validated.
Uploading file to titanic-data/
Uploading an estimated of 3 files
Uploading data/.amlignore
Uploaded data/.amlignore, 1 files out of an estimated total of 3
Uploading data/.amlignore.amltmp
Uploaded data/.amlignore.amltmp, 2 files out of an estimated total of 3
Uploading data/titanic.csv
Uploaded data/titanic.csv, 3 files out of an estimated total of 3
Uploaded 3 files
Creating new dataset


## Provision Compute Cluster for Pipeline

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "sweetdreams"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.
